In [6]:
import sys
sys.path.append("../")

In [7]:
import pandas as pd
import hygia as hg

# Classes instanciations

NOTE: Please check if the model_path matches your path

In [8]:
pre_process_data = hg.PreProcessData()
feature_engineering = hg.FeatureEngineering()
annotate_data = hg.AnnotateData()

new_rf_model = hg.RandomForestModel()

/home/anapaula/dell/Playground/env/lib/python3.8/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/anapaula/dell/Playground/env/lib/python3.8/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Load Data

NOTE: Please check if the file_path matches your data

In [9]:
file_path = '../data/tmp/AI_LATA_ADDRESS_MEX_modificado.csv'
df = pd.read_csv(file_path, sep='¨', nrows=None, engine='python')

# Add new columns

1. Concatenate address
2. All features columns:
    - Key Smash
    - Regex
    - Word Embedding

NOTE: Please check if the columns names matches your data

In [10]:
concatened_column_name = 'concat_STREET_ADDRESS_1_STREET_ADDRESS_2'
pre_process_data.pre_process_data(df, ['STREET_ADDRESS_1', 'STREET_ADDRESS_2'], concatened_column_name)
feature_engineering.extract_features(df, concatened_column_name)

,CUSTOMER_NUM,COMPANY_NUM,ADDRESS_SEQ_NUM,ADDRESS_TYPE,STREET_ADDRESS_1,STREET_ADDRESS_2,CITY,CITY_2,STATE,STATE_L,...,feature_re_only_numbers_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_only_special_characters_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_contains_email_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_contains_url_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_contains_date_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_contains_invalid_words_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_is_substring_of_column_name_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_only_one_char_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_only_white_spaces_concat_STREET_ADDRESS_1_STREET_ADDRESS_2,feature_re_empty_concat_STREET_ADDRESS_1_STREET_ADDRESS_2
0,101551647,3,7,S,CARRET FED 307 KM 298.8,CHETUMAL- PUERTO JUAREZ,PLAYA DEL CARME,N SOLIDARIDAD,MX,QROO,...,False,False,False,False,False,False,False,False,False,False
1,103051972,3,93,S,CARRETERA YERBABUENA-,PUENTECILLA KM. 1.7,GUANAJUATO,,MX,GTO,...,False,False,False,False,False,False,False,False,False,False
2,103051972,3,95,S,CARR IRAPUATO-ABASOLO KM 5,COL PURISIMA DEL JARDIN,IRAPUATO,,MX,GTO,...,False,False,False,False,False,False,False,False,False,False
3,103317515,23,0,B,PRESA DE LAS VIRGENES 115,DESP. 6 COL IRRIGACION DELEG.,MEXICO,,MX,DF,...,False,False,False,False,False,False,False,False,False,False
4,103354057,23,0,S,AV RAFAEL NAVAS SIN NUMERO,COL INDUSTRIAL EL CERRILLO,LERMA,,MX,MEX,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520291,108405112,23,0,S,CORDILLERA DEL MARQUEZ NO 113,,SAN LUIS POTOSI,,MX,SLP,...,False,False,False,False,False,False,False,False,False,False
2520292,104633149,23,0,S,CALLE DE BEISBOL 147,COL. CHURUBUSCO COUNTRY CLUB,DELEGACION COYO,"ACAN, MEXICO",MX,DF,...,False,False,False,False,False,False,False,False,False,False
2520293,112184095,23,0,B,"AV. VICENTE GUERRERO 8951 , CO",L. PARTIDO DOBLADO CD.JUAREZ,CHIHUAHUA,,MX,CHIH,...,False,False,False,False,False,False,False,False,False,False
2520294,108261326,23,0,S,LAGO GARDA 226,AGUA AZUL,NEZAHUALC,OYOTL,MX,MEX,...,False,False,False,False,False,False,False,False,False,False


# Check new columns names

In [11]:
all_features_columns = [col for col in df if col.startswith('feature_ks') or col.startswith('feature_we') or col.startswith('feature_re')]
all_features_columns

['feature_ks_count_sequence_squared_vowels_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_count_sequence_squared_consonants_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_count_sequence_squared_special_characters_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_ratio_of_numeric_digits_squared_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_ks_average_of_char_count_squared_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_0_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_1_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_2_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_3_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_4_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_5_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_6_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_7_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_8_concat_STREET_ADDRESS_1_STREET_ADDRESS_2',
 'feature_we_9_concat_STREET_A

# Annotate data

In [12]:
key_smash_thresholds = {
    'count_sequence_squared_vowels': 1.00,
    'count_sequence_squared_consonants': 1.999,
    'count_sequence_squared_special_characters': 2.2499,
    'ratio_of_numeric_digits_squared': 2.9,
    'average_of_char_count_squared': 2.78,
}

df = annotate_data.annotate_data(df, concatened_column_name, key_smash_thresholds)
df.drop_duplicates(subset=[concatened_column_name])['target'].value_counts()

valid                             1363810
key_smash                             904
contains_email                        412
contains_exactly_the_word_test        182
only_special_characters               151
contains_exactly_the_word_dell        126
only_one_char                          16
contains_invalid_words                 10
only_white_spaces                       2
Name: target, dtype: int64

In [17]:
df['target'].value_counts()

valid                             2514911
key_smash                            1770
only_special_characters              1291
contains_email                        720
contains_exactly_the_word_test        667
contains_exactly_the_word_dell        553
only_one_char                         287
only_white_spaces                      71
contains_invalid_words                 26
Name: target, dtype: int64

# Experiment: retrain model

In [13]:
clf, scores = new_rf_model.train_and_get_scores(df, concatened_column_name, all_features_columns)
scores

array([0.99815498, 1.        , 0.99075786, 0.99260628, 0.97227357])

# Predict using pre-trained model

In [14]:
df['prediction'] = new_rf_model.predict(df[all_features_columns].values)
df.drop_duplicates(subset=[concatened_column_name])['prediction'].value_counts()

/home/anapaula/dell/Playground/env/lib/python3.8/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.0    1361067
1.0       4546
Name: prediction, dtype: int64

# Save predicted data

In [15]:
df[df['prediction'] == 1][[concatened_column_name, 'prediction']].drop_duplicates(subset=[concatened_column_name]).to_csv('../data/tmp/prediction.csv')

In [16]:
df['prediction'] = df['target'] == 'key_smash'
df['prediction_model'] = df['prediction']
df[df['prediction_model']!=df['prediction']][[concatened_column_name, 'target', 'prediction_model', 'prediction']] \
    .drop_duplicates(subset=[concatened_column_name]) \
    .to_csv('../data/tmp/prediction_better_with_model.csv')